In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

%load_ext autoreload
%autoreload 2

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [6]:
data_dir = 'hymenoptera_data'
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x),
                           data_transforms[x]) 
    for x in ['train', 'val']
}

dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
    for x in ['train', 'val']
}

In [8]:
dataset_sizes = {
    x: len(image_datasets[x])
    for x in ['train', 'val']
}

In [9]:
class_names = image_datasets['train'].classes
class_names

['ants', 'bees']

In [10]:
use_gpu = torch.cuda.is_available()

In [14]:
from transfer import train_model

In [15]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/manuel/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:46<00:00, 1015673.14it/s]


In [16]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (b

In [17]:
model_ft.avgpool

AvgPool2d(kernel_size=7, stride=1, padding=0, ceil_mode=False, count_include_pad=True)

In [18]:
if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [20]:
len(list(model_ft.parameters()))

62

In [22]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       dataloaders=dataloaders, dataset_sizes=dataset_sizes,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.5173 Acc: 0.7172
val Loss: 0.2061 Acc: 0.9085

Epoch 1/24
----------
train Loss: 0.5859 Acc: 0.7951
val Loss: 0.4662 Acc: 0.8105

Epoch 2/24
----------
train Loss: 0.3535 Acc: 0.8689
val Loss: 0.2452 Acc: 0.9216

Epoch 3/24
----------
train Loss: 0.4223 Acc: 0.8279
val Loss: 0.3890 Acc: 0.8562

Epoch 4/24
----------
train Loss: 0.4002 Acc: 0.8320


Process Process-43:
Process Process-42:
Process Process-41:
Process Process-44:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/manuel/.pyenv/versions/3.5.2/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/manuel/.pyenv/versions/3.5.2/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/manuel/.pyenv/versions/3.5.2/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/manuel/.pyenv/versions/3.5.2/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/manuel/.pyenv/versions/3.5.2/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/manuel/.pyenv/versions/3.5.2/lib/python3.5/multiprocessing/process.py", line 93, in run
 

KeyboardInterrupt: 